In [1]:
#following libraries to establish connection between user and ssh remote server and also to access sql in server.
import sshtunnel
import mysql.connector
import pandas as pd
import pymysql
import paramiko
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder
from os.path import expanduser
import sys
from pathlib import Path

# code to establish connection with ssh remote server¶

In [30]:
home = expanduser('~')
# mypkey = paramiko.RSAKey.from_private_key_file(home + pkeyfilepath)
# if you want to use ssh password use - ssh_password='your ssh password', bellow
#below are remote ssh & sql variables. 
sql_hostname = '127.0.0.1'
sql_username = 'ifraz-re'
sql_password = '1q2w3e4r'
sql_main_database = 'ghtorrent_201906'
sql_port = 3306
ssh_host = '163.221.172.90'
ssh_user = 'ifraz-re'
ssh_port = 22
ssh_pwd = 'dangerous123'
# below ssh connection code

with SSHTunnelForwarder(
       (ssh_host, ssh_port),
       ssh_username=ssh_user,
       ssh_password=ssh_pwd,
       remote_bind_address=(sql_hostname, sql_port)) as tunnel:
   
 # below open connection with sql @ server thru ssh. 
    conn = pymysql.connect(host='127.0.0.1', user=sql_username,
        passwd=sql_password, db=sql_main_database,
        port=tunnel.local_bind_port)
 # below loading attribute from local csv file.
    file1 = pd.read_csv("gh_login_id_name.csv")
    login_data=file1['login_id']
# below limit the for loop for specific iterations & query to match column between table @ remote server sql and local csv file in a local machine.     

#    limit=10
    j=0   
    return_id=[]
    for i in login_data:
        sys.stdout.write("\rExtracting data {} / {} --> {}".format(j+1, len(login_data), i))
        sys.stdout.flush()

        query = ("select sum(sub.commit_count), count(sub.project_id) as project_count, sub.committer_id from (SELECT COUNT(committer_id) as commit_count,  project_id, committer_id FROM commits WHERE committer_id = '%s' GROUP BY project_id) sub" % i)
        try:
            data = pd.read_sql_query(query, conn)
            return_id.append([data.iloc[0][0],data.iloc[0][1],data.iloc[0][2]])
        except:
            pass
        
        j+=1
#        if j == limit:
#             break
     
    conn.close()

Extracting data 12970 / 12970 --> 32440943

In [31]:
return_id

[[24.0, 3.0, 35401253.0],
 [415.0, 14.0, 41306333.0],
 [12.0, 2.0, 50377777.0],
 [None, 0, None],
 [32.0, 9.0, 2495461.0],
 [77.0, 18.0, 42975307.0],
 [46.0, 12.0, 48886443.0],
 [20.0, 1.0, 51267314.0],
 [33.0, 3.0, 48427628.0],
 [79.0, 12.0, 40912624.0],
 [124.0, 10.0, 9435377.0],
 [7.0, 1.0, 48336897.0],
 [5.0, 1.0, 47064426.0],
 [102.0, 23.0, 48792615.0],
 [22.0, 3.0, 41308115.0],
 [None, 0, None],
 [76.0, 9.0, 33042930.0],
 [6.0, 2.0, 47132764.0],
 [6.0, 3.0, 47182594.0],
 [57.0, 12.0, 11162886.0],
 [3.0, 2.0, 49524328.0],
 [31.0, 11.0, 14946724.0],
 [124.0, 18.0, 32412739.0],
 [14.0, 8.0, 41809547.0],
 [None, 0, None],
 [146.0, 18.0, 48695508.0],
 [None, 0, None],
 [165.0, 11.0, 41137645.0],
 [None, 0, None],
 [4.0, 1.0, 5096118.0],
 [156.0, 21.0, 37740533.0],
 [5.0, 3.0, 49306237.0],
 [33.0, 11.0, 45609678.0],
 [5.0, 4.0, 49450493.0],
 [None, 0, None],
 [11.0, 4.0, 47588107.0],
 [None, 0, None],
 [28.0, 3.0, 36146492.0],
 [None, 0, None],
 [22.0, 3.0, 35148661.0],
 [None, 0, None

In [21]:
mydf= pd.DataFrame(return_id,columns=['commits_sum','total_projects','committer_id'])
mydf.to_csv("commits_projects_summary.csv", index=False)
mydf

,commits_sum,total_projects,committer_id
0,24.0,3.0,35401253.0
1,415.0,14.0,41306333.0
2,12.0,2.0,50377777.0
3,NaN,0.0,NaN
4,32.0,9.0,2495461.0
5,77.0,18.0,42975307.0
6,46.0,12.0,48886443.0
7,20.0,1.0,51267314.0
8,33.0,3.0,48427628.0
9,79.0,12.0,40912624.0
